In [ ]:
import pandas as pd

# 파일 경로 설정
excel_path = 'dart_statements_2015_2024_merged.xlsx'
csv_path = '현금흐름표_간접법_별도_12월.csv'

# 1. 엑셀 파일에서 1번 시트 불러오기
excel_df = pd.read_excel(excel_path, sheet_name=0)

# 2. CSV 파일 전체 불러오기
csv_df = pd.read_csv(csv_path)

# 3. CSV의 결산기준일에서 연도 추출
csv_df['bsns_year'] = pd.to_datetime(csv_df['결산기준일']).dt.year

# 4. 병합 수행: 회사명 + 연도 기준
merged_df = pd.merge(
    excel_df,
    csv_df,
    left_on=['corp_name', 'bsns_year'],
    right_on=['회사명', 'bsns_year'],
    how='inner'  # 또는 'outer', 'left', 'right' 등 필요에 따라
)

# 5. 결과 저장 (선택)
merged_df.to_csv('병합결과.csv', index=False)

# 미리보기
print(merged_df.head())


In [2]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from time import sleep

# 1. .env에서 API 키 로드
load_dotenv()
ECOS_API_KEY = os.getenv("ECOS_API_KEY")

# 2. ECOS 지표 정의
ecos_items = [
    {"stat_code": "060Y001", "cycle": "M", "itm_code": "010101000", "name": "CD_91일"},
    {"stat_code": "200Y002", "cycle": "Q", "itm_code": "AA0101", "name": "실질GDP"},
    {"stat_code": "731Y002", "cycle": "M", "itm_code": "0000001", "name": "환율"},
    {"stat_code": "902Y001", "cycle": "M", "itm_code": "0000001", "name": "CPI"},
    {"stat_code": "901Y001", "cycle": "M", "itm_code": "0000001", "name": "PPI"},
    {"stat_code": "060Y001", "cycle": "M", "itm_code": "010301000", "name": "국고채_3Y"}
]

start_year = 2015
end_year = 2023

all_data = []

# 3. ECOS API 요청
for item in ecos_items:
    url = (
        f"https://ecos.bok.or.kr/api/StatisticSearch/{ECOS_API_KEY}/json/kr/1/10000/"
        f"{item['stat_code']}/{item['cycle']}/{start_year}/{end_year}/{item['itm_code']}"
    )

    try:
        response = requests.get(url)
        sleep(1.5)

        if response.status_code == 200:
            rows = response.json().get("StatisticSearch", {}).get("row", [])
            for row in rows:
                all_data.append({
                    "지표명": item["name"],
                    "시점": row["TIME"],
                    "값": row["DATA_VALUE"]
                })
        else:
            print(f"[{item['name']}] 요청 실패: {response.status_code}")
    except Exception as e:
        print(f"[{item['name']}] 예외 발생: {e}")

# 4. DataFrame으로 변환 및 Excel 저장 (encoding 제거!)
df = pd.DataFrame(all_data)
df.to_excel("ecos_macro_data_2015_2023.xlsx", index=False, engine="openpyxl")

print("✅ 엑셀 저장 완료: ecos_macro_data_2015_2023.xlsx")


✅ 엑셀 저장 완료: ecos_macro_data_2015_2023.xlsx
